In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import HTML, display

- Import the three data sets from FRRP, HICORDIS, and Lamb et al.
- Drop unnecessary columns from species name mapping file
- Trim Lamb et al. dataset down to only diseases
- Drop clade column from FRRP dataset

In [2]:
frrp = pd.read_csv('../input/datasets_renamed/FRRP_renamed.csv')
hicordis = pd.read_csv('../input/datasets_renamed/HICORDIS_renamed.csv')
lamb = pd.read_csv('../input/datasets_renamed/Lamb_et_al_renamed.csv')
spp_name_mapping = pd.read_csv('../input/species_name_mapping.csv')
spp_dict = spp_name_mapping.drop(columns="Specificity")
spp_dict = spp_dict.set_index("Species")['Huang_Roy_tree_name'].to_dict()
lamb = lamb[['Genus','TotalCorals','Total_diseased_Lamb_et_al','Total_compromised_or_bleached_Lamb_et_al','White_syndrome_Lamb_et_al','Black_band_disease_Lamb_et_al','Brown_band_Lamb_et_al','Skeletal_eroding_band_Lamb_et_al','Growth_anomalies_Lamb_et_al']]
frrp.drop(columns="Clade", inplace=True)

- Replace all species names with names found in Huang Roy tree
- Rename Genus column to Species in Lamb et al. dataset

In [3]:
frrp.replace({'Species': spp_dict}, inplace=True)
hicordis.replace({'Species': spp_dict}, inplace=True)
lamb.replace({'Genus': spp_dict}, inplace=True)
lamb.rename(columns={'Genus': 'Species'}, inplace=True)

Set the Species column of each dataset as the index

In [4]:
frrp.set_index("Species", inplace=True)
hicordis.set_index("Species", inplace=True)
lamb.set_index("Species", inplace=True)

Aggregate all species data into one line for each species/genus

In [5]:
frrp = frrp.groupby('Species').agg('sum')
hicordis = hicordis.groupby('Species').agg('mean')
lamb = lamb.groupby('Species').agg('sum')

Divide the number of each disease by the total number of corals to get prevalence rate for each species in the Lamb et al. data, and then drop the TotalCorals column afterward.

In [6]:
total = lamb.TotalCorals
for col in lamb.columns:
    lamb[col] = lamb[col]/total
lamb.drop(columns="TotalCorals", inplace=True)

- First combine the FRRP and HICRODIS datasets
- Second, combine those two datasets with the Lamb et al. data.

In [7]:
f_h_combo = pd.merge(frrp, hicordis, how="outer", left_index = True, right_index = True, indicator=False)
f_h_l_combo = pd.merge(f_h_combo, lamb, how="outer", left_index = True, right_index = True, indicator=False)
fhl_html = f_h_l_combo.to_html()
HTML(fhl_html)

Write table to output folder as a tab seperated values file.

In [8]:
f_h_l_combo.to_csv('../output/merged_disease_table.tsv', sep="\t")